<a href="https://colab.research.google.com/github/thegallier/timeseries/blob/main/timeseries2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
            if torch.rand(1).item() < self.mask_prob:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Define the Model
class OrderModel(nn.Module):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # For timestamp (we don't mask timestamp)
        self.timestamp_linear = nn.Linear(1, embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        # Process the masked sample
        timestamp = x['timestamp'].unsqueeze(1).float()
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        timestamp_emb = self.timestamp_linear(timestamp)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([timestamp_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings
        batch_size = timestamp.shape[0]

        seq_timestamp = seq_data['timestamp'].float()
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_timestamp_emb = self.timestamp_linear(seq_timestamp.unsqueeze(-1))  # (batch_size, seq_len, embedding_dim)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_timestamp_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = nn.utils.rnn.pack_padded_sequence(seq_emb, seq_lengths, batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n.squeeze(0)  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Collate function for DataLoader
def collate_fn(batch):
    # batch is a list of (masked_sample, target, sequence)
    batch_size = len(batch)
    masked_samples = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    sequences = [item[2] for item in batch]

    # Convert masked_samples to tensors
    batch_data = {}
    batch_target = {}
    seq_data = {}
    seq_lengths = []

    # Process data
    for key in ['timestamp', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_data[key] = torch.tensor([sample[key] for sample in masked_samples])

    # Process sequences
    # Sequences are variable-length
    # For each key, we have a list of sequences
    for key in ['timestamp', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        seq_list = [torch.tensor(seq[key]) for seq in sequences]
        seq_padded = nn.utils.rnn.pad_sequence(seq_list, batch_first=True, padding_value=0)
        seq_data[key] = seq_padded
    # Record lengths
    seq_lengths = [len(seq['timestamp']) for seq in sequences]

    # Convert seq_lengths to tensor
    seq_lengths = torch.tensor(seq_lengths, dtype=torch.long)

    # Process target (exclude 'timestamp' as it's not in target)
    for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_target[key] = []
        for target in targets:
            if target[key] is not None:
                batch_target[key].append(target[key])
            else:
                batch_target[key].append(-100 if key not in ['quantity', 'price'] else 0.0)

        # Convert to tensor and create mask
        if key in ['quantity', 'price']:
            # For continuous features
            batch_target[key + '_mask'] = torch.tensor([t != 0.0 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.float)
        else:
            # For categorical features
            batch_target[key + '_mask'] = torch.tensor([t != -100 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.long)

    return batch_data, batch_target, seq_data, seq_lengths

# Training and Testing Loop
def train_model():
    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(num_samples=10000)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

    # Create model
    model = OrderModel(N_sec_ids=dataset.N_sec_ids, N_distance_bins=dataset.N_distance_bins)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Loss functions
    criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
    criterion_l1 = nn.L1Loss()

    # Training loop
    for epoch in range(5):
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for security_id
            target = batch_target['security_id']
            mask = batch_target['security_id_mask']
            if mask.any():
                loss_sec_id = criterion_ce(outputs['security_id'], target)
                loss += loss_sec_id

            # Compute loss for buy_sell
            target = batch_target['buy_sell']
            mask = batch_target['buy_sell_mask']
            if mask.any():
                loss_buy_sell = criterion_ce(outputs['buy_sell'], target)
                loss += loss_buy_sell

            # Compute loss for add_modify_delete
            target = batch_target['add_modify_delete']
            mask = batch_target['add_modify_delete_mask']
            if mask.any():
                loss_amd = criterion_ce(outputs['add_modify_delete'], target)
                loss += loss_amd

            # Compute loss for distance
            target = batch_target['distance']
            mask = batch_target['distance_mask']
            if mask.any():
                loss_distance = criterion_ce(outputs['distance'], target)
                loss += loss_distance

            # Compute loss for quantity
            target = batch_target['quantity']
            mask = batch_target['quantity_mask']
            if mask.any():
                output = outputs['quantity'].squeeze()
                loss_quantity = criterion_l1(output[mask], target[mask])
                loss += loss_quantity

            # Compute loss for price
            target = batch_target['price']
            mask = batch_target['price_mask']
            if mask.any():
                output = outputs['price'].squeeze()
                loss_price = criterion_l1(output[mask], target[mask])
                loss += loss_price

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

if __name__ == "__main__":
    train_model()

<ipython-input-2-d57e3d80251b>:238: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 365.27560881596463
Epoch 2, Loss: 327.6048114557934
Epoch 3, Loss: 325.116089061567
Epoch 4, Loss: 312.7274585893959
Epoch 5, Loss: 312.8242979596375


In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
            'position': torch.arange(seq_len)  # Row number
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
            if torch.rand(1).item() < self.mask_prob:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Learnable Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(PositionalEncoder, self).__init__()
        self.timestamp_encoder = nn.Linear(1, embedding_dim)
        self.position_encoder = nn.Embedding(1000, embedding_dim)  # Assume max 1000 positions

    def forward(self, timestamps, positions):
        timestamp_emb = self.timestamp_encoder(timestamps.unsqueeze(-1).float())
        position_emb = self.position_encoder(positions)
        return timestamp_emb + position_emb

# Base Model Class
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, *args, **kwargs):
        raise NotImplementedError

# Order Model with Positional Encoder
class OrderModel(BaseModel):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # Positional Encoder
        self.positional_encoder = PositionalEncoder(embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        batch_size = x['timestamp'].shape[0]

        # Process the masked sample
        timestamp = x['timestamp']
        position = x['position']
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        positional_emb = self.positional_encoder(timestamp, position)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([positional_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings

        seq_timestamp = seq_data['timestamp']
        seq_position = seq_data['position']
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_positional_emb = self.positional_encoder(seq_timestamp, seq_position)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_positional_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_emb, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Additional Models

class MatrixRegressionModel(BaseModel):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, num_securities, input_size, num_classes=1):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(1, num_securities))
        self.B = nn.Parameter(torch.randn(input_size, num_classes))

    def forward(self, x, *args, **kwargs):
        # x: (batch_size, input_size)
        x = x.view(x.size(0), -1)  # Flatten over time
        out = self.A @ x @ self.B  # Shape: (batch_size, num_classes)
        return {'price': out.squeeze(-1)}  # Adjust as needed

class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, num_classes=1):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x, *args, **kwargs):
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        return {'price': out.squeeze(-1)}  # Adjust as needed

class CNNModel(BaseModel):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, num_features, num_classes=1):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x, *args, **kwargs):
        x = x.permute(0, 2, 1)  # (batch_size, num_features, seq_len)
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)
        return {'price': x.squeeze(-1)}  # Adjust as needed

class LSTMModel(BaseModel):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, num_classes=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x, seq_data, seq_lengths):
        # x: Not used in this model
        # seq_data: input sequences
        # seq_lengths: lengths of sequences

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_data, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        h0 = torch.zeros(self.lstm.num_layers, seq_data.size(0), self.lstm.hidden_size).to(seq_data.device)
        c0 = torch.zeros(self.lstm.num_layers, seq_data.size(0), self.lstm.hidden_size).to(seq_data.device)

        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb, (h0, c0))

        # Get the last hidden state
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_size)

        out = self.fc(seq_context)
        return {'price': out.squeeze(-1)}  # Adjust as needed

# Collate function for DataLoader
def collate_fn(batch):
    # batch is a list of (masked_sample, target, sequence)
    batch_size = len(batch)
    masked_samples = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    sequences = [item[2] for item in batch]

    # Convert masked_samples to tensors
    batch_data = {}
    batch_target = {}
    seq_data = {}
    seq_lengths = []

    # Process data
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_data[key] = torch.tensor([sample[key] for sample in masked_samples])

    # Process sequences
    # Sequences are variable-length
    # For each key, we have a list of sequences
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        seq_list = [torch.tensor(seq[key]) for seq in sequences]
        seq_padded = nn.utils.rnn.pad_sequence(seq_list, batch_first=True, padding_value=0)
        seq_data[key] = seq_padded
    # Record lengths
    seq_lengths = [len(seq['timestamp']) for seq in sequences]

    # Convert seq_lengths to tensor
    seq_lengths = torch.tensor(seq_lengths, dtype=torch.long)

    # Process target (exclude 'timestamp' and 'position' as they're not in target)
    for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_target[key] = []
        for target in targets:
            if target[key] is not None:
                batch_target[key].append(target[key])
            else:
                batch_target[key].append(-100 if key not in ['quantity', 'price'] else 0.0)

        # Convert to tensor and create mask
        if key in ['quantity', 'price']:
            # For continuous features
            batch_target[key + '_mask'] = torch.tensor([t != 0.0 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.float)
        else:
            # For categorical features
            batch_target[key + '_mask'] = torch.tensor([t != -100 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.long)

    return batch_data, batch_target, seq_data, seq_lengths

# Training and Testing Loop
def train_model(model_class, model_params, dataset_params, training_params):
    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    # Create model
    model = model_class(**model_params)
    optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

    # Loss functions
    criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
    criterion_l1 = nn.L1Loss()

    # Training loop
    for epoch in range(training_params['num_epochs']):
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for security_id
            if 'security_id' in outputs:
                target = batch_target['security_id']
                mask = batch_target['security_id_mask']
                if mask.any():
                    loss_sec_id = criterion_ce(outputs['security_id'], target)
                    loss += loss_sec_id

            # Compute loss for buy_sell
            if 'buy_sell' in outputs:
                target = batch_target['buy_sell']
                mask = batch_target['buy_sell_mask']
                if mask.any():
                    loss_buy_sell = criterion_ce(outputs['buy_sell'], target)
                    loss += loss_buy_sell

            # Compute loss for add_modify_delete
            if 'add_modify_delete' in outputs:
                target = batch_target['add_modify_delete']
                mask = batch_target['add_modify_delete_mask']
                if mask.any():
                    loss_amd = criterion_ce(outputs['add_modify_delete'], target)
                    loss += loss_amd

            # Compute loss for distance
            if 'distance' in outputs:
                target = batch_target['distance']
                mask = batch_target['distance_mask']
                if mask.any():
                    loss_distance = criterion_ce(outputs['distance'], target)
                    loss += loss_distance

            # Compute loss for quantity
            if 'quantity' in outputs:
                target = batch_target['quantity']
                mask = batch_target['quantity_mask']
                if mask.any():
                    output = outputs['quantity'].squeeze()
                    loss_quantity = criterion_l1(output[mask], target[mask])
                    loss += loss_quantity

            # Compute loss for price
            if 'price' in outputs:
                target = batch_target['price']
                mask = batch_target['price_mask']
                if mask.any():
                    output = outputs['price'].squeeze()
                    loss_price = criterion_l1(output[mask], target[mask])
                    loss += loss_price

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

    return model

# Hyperparameter Optimization
def hyperparameter_optimization():
    from sklearn.model_selection import GridSearchCV
    from skorch import NeuralNetRegressor, NeuralNetClassifier
    import numpy as np

    # Define parameter grid
    param_grid = {
        'module__embedding_dim': [16, 32],
        'module__hidden_dim': [32, 64],
        'optimizer__lr': [0.001, 0.0001],
    }

    # Wrap the model using skorch
    net = NeuralNetRegressor(
        OrderModel,
        module__N_sec_ids=100,
        module__N_distance_bins=10,
        max_epochs=5,
        lr=0.001,
        iterator_train__collate_fn=collate_fn,
        iterator_valid__collate_fn=collate_fn,
        device='cuda' if torch.cuda.is_available() else 'cpu',
    )

    # Create Dataset
    dataset = MaskedOrderDataset(num_samples=1000)

    # Prepare data for GridSearchCV (this is a simplified example)
    X = [dataset[i][0] for i in range(len(dataset))]
    y = [dataset[i][1]['price'] if dataset[i][1]['price'] is not None else 0.0 for i in range(len(dataset))]

    # Convert X and y to numpy arrays (skorch expects numpy arrays)
    X = np.array(X)
    y = np.array(y)

    # Perform Grid Search
    gs = GridSearchCV(net, param_grid, refit=False, cv=3, scoring='neg_mean_absolute_error', verbose=2)
    gs.fit(X, y)

    print("Best parameters found:")
    print(gs.best_params_)

    print("Best score:")
    print(gs.best_score_)

# Example Usage
if __name__ == "__main__":
    # Training parameters
    training_params = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'num_epochs': 5,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }

    # Dataset parameters
    dataset_params = {
        'num_samples': 5000,
        'mask_prob': 0.15,
        'max_seq_len': 50
    }

    # Model parameters for OrderModel
    model_params = {
        'N_sec_ids': 100,
        'N_distance_bins': 10,
        'embedding_dim': 32,
        'hidden_dim': 64
    }

    # Train OrderModel
    print("Training OrderModel...")
    model = train_model(OrderModel, model_params, dataset_params, training_params)

    # You can substitute different models by changing model_class and model_params
    # For example, training LSTMModel
    '''
    print("Training LSTMModel...")
    model_params_lstm = {
        'input_size': 7,  # Number of features in sequence data
        'hidden_size': 64,
        'num_layers': 2,
        'num_classes': 1
    }
    model = train_model(LSTMModel, model_params_lstm, dataset_params, training_params)
    '''

    # Hyperparameter Optimization
    '''
    print("Starting Hyperparameter Optimization...")
    hyperparameter_optimization()
    '''

Training OrderModel...


<ipython-input-1-15b0f17eec53>:371: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 381.8814974256382
Epoch 2, Loss: 349.19875107297474
Epoch 3, Loss: 338.50678306628186
Epoch 4, Loss: 321.11537005187597
Epoch 5, Loss: 339.4471651308096


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random
from collections import defaultdict
import numpy as np

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
            'position': torch.arange(seq_len)  # Row number
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        # List of features to potentially mask
        maskable_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']

        # Decide how many features to mask (at least one)
        num_features_to_mask = random.randint(1, len(maskable_keys))

        # Randomly select features to mask
        features_to_mask = random.sample(maskable_keys, num_features_to_mask)

        for key in maskable_keys:
            if key in features_to_mask:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Learnable Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(PositionalEncoder, self).__init__()
        self.timestamp_encoder = nn.Linear(1, embedding_dim)
        self.position_encoder = nn.Embedding(1000, embedding_dim)  # Assume max 1000 positions

    def forward(self, timestamps, positions):
        timestamp_emb = self.timestamp_encoder(timestamps.unsqueeze(-1).float())
        position_emb = self.position_encoder(positions)
        return timestamp_emb + position_emb

# Base Model Class
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, *args, **kwargs):
        raise NotImplementedError

# Order Model with Positional Encoder
class OrderModel(BaseModel):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # Positional Encoder
        self.positional_encoder = PositionalEncoder(embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        batch_size = x['timestamp'].shape[0]

        # Process the masked sample
        timestamp = x['timestamp']
        position = x['position']
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        positional_emb = self.positional_encoder(timestamp, position)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([positional_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings

        seq_timestamp = seq_data['timestamp']
        seq_position = seq_data['position']
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_positional_emb = self.positional_encoder(seq_timestamp, seq_position)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_positional_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_emb, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Additional Models (Unchanged for brevity)
# Assume the BaseModel class and other models are defined as before

# Collate function for DataLoader
def collate_fn(batch):
    # batch is a list of (masked_sample, target, sequence)
    batch_size = len(batch)
    masked_samples = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    sequences = [item[2] for item in batch]

    # Convert masked_samples to tensors
    batch_data = {}
    batch_target = {}
    seq_data = {}
    seq_lengths = []

    # Process data
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_data[key] = torch.tensor([sample[key] for sample in masked_samples])

    # Process sequences
    # Sequences are variable-length
    # For each key, we have a list of sequences
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        seq_list = [torch.tensor(seq[key]) for seq in sequences]
        seq_padded = nn.utils.rnn.pad_sequence(seq_list, batch_first=True, padding_value=0)
        seq_data[key] = seq_padded
    # Record lengths
    seq_lengths = [len(seq['timestamp']) for seq in sequences]

    # Convert seq_lengths to tensor
    seq_lengths = torch.tensor(seq_lengths, dtype=torch.long)

    # Process target (exclude 'timestamp' and 'position' as they're not in target)
    for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_target[key] = []
        for target in targets:
            if target[key] is not None:
                batch_target[key].append(target[key])
            else:
                batch_target[key].append(-100 if key not in ['quantity', 'price'] else 0.0)

        # Convert to tensor and create mask
        if key in ['quantity', 'price']:
            # For continuous features
            batch_target[key + '_mask'] = torch.tensor([t != 0.0 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.float)
        else:
            # For categorical features
            batch_target[key + '_mask'] = torch.tensor([t != -100 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.long)

    return batch_data, batch_target, seq_data, seq_lengths

# Training and Testing Loop
def train_model(model_class, model_params, dataset_params, training_params):
    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    # Create model
    model = model_class(**model_params).to(training_params['device'])
    optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

    # Loss functions
    criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
    criterion_l1 = nn.L1Loss()

    # Training loop
    for epoch in range(training_params['num_epochs']):
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for each feature
            for key in ['security_id', 'buy_sell', 'add_modify_delete', 'distance']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        loss_feature = criterion_ce(outputs[key], target)
                        loss += loss_feature

            for key in ['quantity', 'price']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key].squeeze()
                        loss_feature = criterion_l1(output[mask], target[mask])
                        loss += loss_feature

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

    return model

# Beam Search Generation Function
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random
from collections import defaultdict
import numpy as np

# [The code above remains unchanged until the generate_sequences function]

# Beam Search Generation Function
def generate_sequences(model, past_data, security_id, num_timesteps, beam_width, num_samples, device='cpu'):
    """
    Generate sequences using beam search.

    Args:
        model: Trained model.
        past_data: Dictionary containing past sequences.
        security_id: The security ID to generate sequences for.
        num_timesteps: Number of timesteps to generate.
        beam_width: Beam width.
        num_samples: Number of samples to generate.
        device: Device to run the computations on.

    Returns:
        generated_sequences: A list of generated sequences.
    """
    model.eval()
    with torch.no_grad():
        # Initialize the beam with the past_data
        # Each beam entry is a tuple (sequence, cumulative_log_prob)
        beam = [({'timestamp': past_data['timestamp'],
                  'position': past_data['position'],
                  'security_id': past_data['security_id'],
                  'buy_sell': past_data['buy_sell'],
                  'add_modify_delete': past_data['add_modify_delete'],
                  'quantity': past_data['quantity'],
                  'price': past_data['price'],
                  'distance': past_data['distance']},
                 0.0)]  # Start with log probability 0

        for t in range(num_timesteps):
            new_beam = []
            for seq, cum_log_prob in beam:
                # Prepare input data
                # Use the last max_seq_len elements
                seq_len = len(seq['timestamp'])
                start_idx = max(0, seq_len - model.lstm_input_dim)
                seq_data = {key: torch.tensor(seq[key][start_idx:]).unsqueeze(0).to(device) for key in seq}
                seq_lengths = torch.tensor([seq_data['timestamp'].shape[1]]).to(device)

                # Create a sample with masked features (except security_id and timestamp)
                sample = {
                    'timestamp': torch.tensor([seq['timestamp'][-1] + 1]).to(device),
                    'position': torch.tensor([seq['position'][-1] + 1]).to(device),
                    'security_id': torch.tensor([security_id]).to(device),
                    'buy_sell': torch.tensor([model.buy_sell_mask]).to(device),
                    'add_modify_delete': torch.tensor([model.add_modify_delete_mask]).to(device),
                    'quantity': torch.tensor([0.0]).to(device),
                    'price': torch.tensor([0.0]).to(device),
                    'distance': torch.tensor([model.distance_mask]).to(device)
                }

                # Predict missing features
                outputs = model(sample, seq_data, seq_lengths)

                # For each categorical feature, get top N predictions
                candidates = []

                # Adjust topk for buy_sell
                buy_sell_probs = F.softmax(outputs['buy_sell'], dim=-1)
                num_buy_sell_classes = buy_sell_probs.size(-1)
                buy_sell_topk_k = min(beam_width, num_buy_sell_classes)
                buy_sell_topk = torch.topk(buy_sell_probs, buy_sell_topk_k)

                # Adjust topk for add_modify_delete
                add_modify_delete_probs = F.softmax(outputs['add_modify_delete'], dim=-1)
                num_amd_classes = add_modify_delete_probs.size(-1)
                amd_topk_k = min(beam_width, num_amd_classes)
                amd_topk = torch.topk(add_modify_delete_probs, amd_topk_k)

                # Adjust topk for distance
                distance_probs = F.softmax(outputs['distance'], dim=-1)
                num_distance_classes = distance_probs.size(-1)
                distance_topk_k = min(beam_width, num_distance_classes)
                distance_topk = torch.topk(distance_probs, distance_topk_k)

                # For continuous features, we can use the predicted value directly
                quantity_pred = outputs['quantity'].item()
                price_pred = outputs['price'].item()

                # Generate combinations of top predictions
                for i in range(buy_sell_topk_k):
                    for j in range(amd_topk_k):
                        for k in range(distance_topk_k):
                            new_seq = {key: seq[key] + [sample[key].item()] for key in seq}
                            new_seq['buy_sell'][-1] = buy_sell_topk.indices[0][i].item()
                            new_seq['add_modify_delete'][-1] = amd_topk.indices[0][j].item()
                            new_seq['distance'][-1] = distance_topk.indices[0][k].item()
                            new_seq['quantity'][-1] = quantity_pred
                            new_seq['price'][-1] = price_pred

                            # Compute new cumulative log probability
                            log_prob = cum_log_prob + \
                                       torch.log(buy_sell_topk.values[0][i] + 1e-9).item() + \
                                       torch.log(amd_topk.values[0][j] + 1e-9).item() + \
                                       torch.log(distance_topk.values[0][k] + 1e-9).item()
                            candidates.append((new_seq, log_prob))

                # Keep top beam_width candidates
                candidates.sort(key=lambda x: x[1], reverse=True)
                new_beam.extend(candidates[:beam_width])

            # Keep top beam_width sequences
            new_beam.sort(key=lambda x: x[1], reverse=True)
            beam = new_beam[:beam_width]

        # After generation, sample from the beam to get the required number of samples
        generated_sequences = [seq for seq, _ in beam[:num_samples]]
        return generated_sequences


# Example Usage
if __name__ == "__main__":
    # Training parameters
    training_params = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'num_epochs': 5,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }

    # Dataset parameters
    dataset_params = {
        'num_samples': 5000,
        'mask_prob': 0.15,
        'max_seq_len': 50
    }

    # Model parameters for OrderModel
    model_params = {
        'N_sec_ids': 100,
        'N_distance_bins': 10,
        'embedding_dim': 32,
        'hidden_dim': 64
    }

    # Train OrderModel
    print("Training OrderModel...")
    model = train_model(OrderModel, model_params, dataset_params, training_params)

    # Generate sequences using beam search
    print("\nGenerating sequences using beam search...")
    # Prepare past_data
    past_data = {
        'timestamp': [1000, 1001, 1002],
        'position': [0, 1, 2],
        'security_id': [5, 5, 5],
        'buy_sell': [0, 1, 0],
        'add_modify_delete': [0, 1, 2],
        'quantity': [500.0, 600.0, 700.0],
        'price': [50.0, 51.0, 52.0],
        'distance': [3, 2, 1]
    }

    # Parameters for generation
    security_id = 5
    num_timesteps = 5
    beam_width = 3
    num_samples = 100

    generated_sequences = generate_sequences(
        model=model,
        past_data=past_data,
        security_id=security_id,
        num_timesteps=num_timesteps,
        beam_width=beam_width,
        num_samples=num_samples,
        device=training_params['device']
    )

    # Compute distribution of buys vs sells
    total_buy = 0
    total_sell = 0
    total_actions = 0

    for seq in generated_sequences:
        buy_sell_seq = seq['buy_sell'][-num_timesteps:]  # Only consider generated timesteps
        total_buy += buy_sell_seq.count(0)
        total_sell += buy_sell_seq.count(1)
        total_actions += len(buy_sell_seq)

    buy_percentage = (total_buy / total_actions) * 100 if total_actions > 0 else 0
    sell_percentage = (total_sell / total_actions) * 100 if total_actions > 0 else 0

    print(f"\nGenerated {num_samples} sequences of {num_timesteps} timesteps each.")
    print(f"Buy actions: {buy_percentage:.2f}%")
    print(f"Sell actions: {sell_percentage:.2f}%")

Training OrderModel...


<ipython-input-3-fa16ee2492a3>:279: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 375.6108235158738
Epoch 2, Loss: 349.5281865794188
Epoch 3, Loss: 321.91743964905953
Epoch 4, Loss: 304.81262012651774
Epoch 5, Loss: 295.27519867526496

Generating sequences using beam search...

Generated 3 sequences of 5 timesteps each.
Buy actions: 40.00%
Sell actions: 60.00%


In [6]:
    # Parameters for generation
    security_id = 5
    num_timesteps = 50
    beam_width = 45
    num_samples = 130

    generated_sequences = generate_sequences(
        model=model,
        past_data=past_data,
        security_id=security_id,
        num_timesteps=num_timesteps,
        beam_width=beam_width,
        num_samples=num_samples,
        device=training_params['device']
    )

    # Compute distribution of buys vs sells
    total_buy = 0
    total_sell = 0
    total_actions = 0

    for seq in generated_sequences:
        buy_sell_seq = seq['buy_sell'][-num_timesteps:]  # Only consider generated timesteps
        total_buy += buy_sell_seq.count(0)
        total_sell += buy_sell_seq.count(1)
        total_actions += len(buy_sell_seq)

    buy_percentage = (total_buy / total_actions) * 100 if total_actions > 0 else 0
    sell_percentage = (total_sell / total_actions) * 100 if total_actions > 0 else 0

    print(f"\nGenerated {num_samples} sequences of {num_timesteps} timesteps each.")
    print(f"Buy actions: {buy_percentage:.2f}%")
    print(f"Sell actions: {sell_percentage:.2f}%")


Generated 130 sequences of 50 timesteps each.
Buy actions: 91.42%
Sell actions: 8.58%


In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random
from collections import defaultdict
import numpy as np

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
            'position': torch.arange(seq_len)  # Row number
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        # List of features to potentially mask
        maskable_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']

        # Decide how many features to mask (at least one)
        num_features_to_mask = random.randint(1, len(maskable_keys))

        # Randomly select features to mask
        features_to_mask = random.sample(maskable_keys, num_features_to_mask)

        for key in maskable_keys:
            if key in features_to_mask:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Learnable Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(PositionalEncoder, self).__init__()
        self.timestamp_encoder = nn.Linear(1, embedding_dim)
        self.position_encoder = nn.Embedding(1000, embedding_dim)  # Assume max 1000 positions

    def forward(self, timestamps, positions):
        timestamp_emb = self.timestamp_encoder(timestamps.unsqueeze(-1).float())
        position_emb = self.position_encoder(positions)
        return timestamp_emb + position_emb

# Base Model Class
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, *args, **kwargs):
        raise NotImplementedError

# Order Model with Positional Encoder
class OrderModel(BaseModel):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # Positional Encoder
        self.positional_encoder = PositionalEncoder(embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        batch_size = x['timestamp'].shape[0]

        # Process the masked sample
        timestamp = x['timestamp']
        position = x['position']
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        positional_emb = self.positional_encoder(timestamp, position)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([positional_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings

        seq_timestamp = seq_data['timestamp']
        seq_position = seq_data['position']
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_positional_emb = self.positional_encoder(seq_timestamp, seq_position)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_positional_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_emb, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Additional Models

class MatrixRegressionModel(BaseModel):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, input_size, output_keys):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(input_size, input_size))
        self.B = nn.Parameter(torch.randn(input_size, len(output_keys)))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        out = x_vector @ self.A @ self.B  # Shape: (batch_size, num_outputs)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = out[:, i]

        return outputs

class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, output_keys):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        out = self.linear(x_vector)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = out[:, i]

        return outputs

class CNNModel(BaseModel):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, input_size, output_keys):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        seq_vector = seq_vector.permute(0, 2, 1)  # (batch_size, num_features, seq_len)
        x = self.relu(self.conv1(seq_vector))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = x[:, i]

        return outputs

class LSTMModel(BaseModel):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, output_keys):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        packed_seq_emb = pack_padded_sequence(seq_vector, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        h0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)
        c0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)

        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb, (h0, c0))

        # Get the last hidden state
        seq_context = h_n[-1​⬤

SyntaxError: invalid decimal literal (<ipython-input-7-9eb5d33bcc7f>, line 377)

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random
from collections import defaultdict
import numpy as np

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
            'position': torch.arange(seq_len)  # Row number
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        # List of features to potentially mask
        maskable_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']

        # Decide how many features to mask (at least one)
        num_features_to_mask = random.randint(1, len(maskable_keys))

        # Randomly select features to mask
        features_to_mask = random.sample(maskable_keys, num_features_to_mask)

        for key in maskable_keys:
            if key in features_to_mask:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Learnable Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(PositionalEncoder, self).__init__()
        self.timestamp_encoder = nn.Linear(1, embedding_dim)
        self.position_encoder = nn.Embedding(1000, embedding_dim)  # Assume max 1000 positions

    def forward(self, timestamps, positions):
        timestamp_emb = self.timestamp_encoder(timestamps.unsqueeze(-1).float())
        position_emb = self.position_encoder(positions)
        return timestamp_emb + position_emb

# Base Model Class
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, *args, **kwargs):
        raise NotImplementedError

# Order Model with Positional Encoder
class OrderModel(BaseModel):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # Positional Encoder
        self.positional_encoder = PositionalEncoder(embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        batch_size = x['timestamp'].shape[0]

        # Process the masked sample
        timestamp = x['timestamp']
        position = x['position']
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        positional_emb = self.positional_encoder(timestamp, position)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([positional_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings

        seq_timestamp = seq_data['timestamp']
        seq_position = seq_data['position']
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_positional_emb = self.positional_encoder(seq_timestamp, seq_position)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_positional_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_emb, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Additional Models

class MatrixRegressionModel(BaseModel):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, input_size, output_keys):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(input_size, input_size))
        self.B = nn.Parameter(torch.randn(input_size, len(output_keys)))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        out = x_vector @ self.A @ self.B  # Shape: (batch_size, num_outputs)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = out[:, i]

        return outputs

class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, output_keys):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        out = self.linear(x_vector)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = out[:, i]

        return outputs

class CNNModel(BaseModel):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, input_size, output_keys):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(128, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        seq_vector = seq_vector.permute(0, 2, 1)  # (batch_size, num_features, seq_len)
        x = self.relu(self.conv1(seq_vector))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling
        x = self.fc(x)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = x[:, i]

        return outputs

class LSTMModel(BaseModel):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, output_keys):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        packed_seq_emb = pack_padded_sequence(seq_vector, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        h0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)
        c0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)

        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb, (h0, c0))

        # Get the last hidden state
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_size)

        x = self.fc(seq_context)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = x[:, i]

        return outputs

class TransformerModel(BaseModel):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallelizable.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.
    """
    def __init__(self, input_size, num_heads, num_layers, output_keys):
        super(TransformerModel, self).__init__()
        self.positional_encoder = PositionalEncoder(input_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_size, len(output_keys))
        self.output_keys = output_keys

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() if key in ['quantity', 'price'] else seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        # Add positional encoding
        positions = torch.arange(0, seq_vector.size(1), device=seq_vector.device).unsqueeze(0).repeat(seq_vector.size(0), 1)
        seq_vector = seq_vector + self.positional_encoder(positions, positions)

        # Transformer expects input shape (seq_len, batch_size, feature_dim)
        seq_vector = seq_vector.permute(1, 0, 2)
        transformer_output = self.transformer_encoder(seq_vector)
        # Take the output corresponding to the last position
        x = transformer_output[-1, :, :]
        x = self.fc(x)

        outputs = {}
        for i, key in enumerate(self.output_keys):
            outputs[key] = x[:, i]

        return outputs

# Collate function for DataLoader
def collate_fn(batch):
    # batch is a list of (masked_sample, target, sequence)
    batch_size = len(batch)
    masked_samples = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    sequences = [item[2] for item in batch]

    # Convert masked_samples to tensors
    batch_data = {}
    batch_target = {}
    seq_data = {}
    seq_lengths = []

    # Process data
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_data[key] = torch.tensor([sample[key] for sample in masked_samples])

    # Process sequences
    # Sequences are variable-length
    # For each key, we have a list of sequences
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        seq_list = [torch.tensor(seq[key]) for seq in sequences]
        seq_padded = nn.utils.rnn.pad_sequence(seq_list, batch_first=True, padding_value=0)
        seq_data[key] = seq_padded
    # Record lengths
    seq_lengths = [len(seq['timestamp']) for seq in sequences]

    # Convert seq_lengths to tensor
    seq_lengths = torch.tensor(seq_lengths, dtype=torch.long)

    # Process target (exclude 'timestamp' and 'position' as they're not in target)
    for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_target[key] = []
        for target in targets:
            if target[key] is not None:
                batch_target[key].append(target[key])
            else:
                batch_target[key].append(-100 if key not in ['quantity', 'price'] else 0.0)

        # Convert to tensor and create mask
        if key in ['quantity', 'price']:
            # For continuous features
            batch_target[key + '_mask'] = torch.tensor([t != 0.0 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.float)
        else:
            # For categorical features
            batch_target[key + '_mask'] = torch.tensor([t != -100 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.long)

    return batch_data, batch_target, seq_data, seq_lengths

# Training and Testing Loop
def train_model(model_class, model_params, dataset_params, training_params):
    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    # Create model
    model = model_class(**model_params).to(training_params['device'])
    optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

    # Loss functions
    criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
    criterion_l1 = nn.L1Loss()

    # Training loop
    for epoch in range(training_params['num_epochs']):
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for each feature
            for key in ['security_id', 'buy_sell', 'add_modify_delete', 'distance']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        loss_feature = criterion_ce(outputs[key], target)
                        loss += loss_feature

            for key in ['quantity', 'price']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key].squeeze()
                        loss_feature = criterion_l1(output[mask], target[mask])
                        loss += loss_feature

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

    return model

# Beam Search Generation Function
def generate_sequences(model, past_data, security_id, num_timesteps, beam_width, num_samples, device='cpu'):
    """
    Generate sequences using beam search.

    Args:
        model: Trained model.
        past_data: Dictionary containing past sequences.
        security_id: The security ID to generate sequences for.
        num_timesteps: Number of timesteps to generate.
        beam_width: Beam width.
        num_samples: Number of samples to generate.
        device: Device to run the computations on.

    Returns:
        generated_sequences: A list of generated sequences.
    """
    model.eval()
    with torch.no_grad():
        # Initialize the beam with the past_data
        # Each beam entry is a tuple (sequence, cumulative_log_prob)
        beam = [({'timestamp': past_data['timestamp'],
                  'position': past_data['position'],
                  'security_id': past_data['security_id'],
                  'buy_sell': past_data['buy_sell'],
                  'add_modify_delete': past_data['add_modify_delete'],
                  'quantity': past_data['quantity'],
                  'price': past_data['price'],
                  'distance': past_data['distance']},
                 0.0)]  # Start with log probability 0

        for t in range(num_timesteps):
            new_beam = []
            for seq, cum_log_prob in beam:
                # Prepare input data
                # Use the last max_seq_len elements
                seq_len = len(seq['timestamp'])
                start_idx = max(0, seq_len - model.lstm_input_dim) if hasattr(model, 'lstm_input_dim') else 0
                seq_data = {key: torch.tensor(seq[key][start_idx:]).unsqueeze(0).to(device) for key in seq}
                seq_lengths = torch.tensor([seq_data['timestamp'].shape[1]]).to(device)

                # Create a sample with masked features (except security_id and timestamp)
                sample = {
                    'timestamp': torch.tensor([seq['timestamp'][-1] + 1]).to(device),
                    'position': torch.tensor([seq['position'][-1] + 1]).to(device),
                    'security_id': torch.tensor([security_id]).to(device),
                    'buy_sell': torch.tensor([model.buy_sell_mask if hasattr(model, 'buy_sell_mask') else 2]).to(device),
                    'add_modify_delete': torch.tensor([model.add_modify_delete_mask if hasattr(model, 'add_modify_delete_mask') else 3]).to(device),
                    'quantity': torch.tensor([0.0]).to(device),
                    'price': torch.tensor([0.0]).to(device),
                    'distance': torch.tensor([model.distance_mask if hasattr(model, 'distance_mask') else 10]).to(device)
                }

                # Predict missing features
                outputs = model(sample, seq_data, seq_lengths)

                # For each categorical feature, get top N predictions
                candidates = []

                # Adjust topk for buy_sell
                if 'buy_sell' in outputs:
                    buy_sell_probs = F.softmax(outputs['buy_sell'], dim=-1)
                    num_buy_sell_classes = buy_sell_probs.size(-1)
                    buy_sell_topk_k = min(beam_width, num_buy_sell_classes)
                    buy_sell_topk = torch.topk(buy_sell_probs, buy_sell_topk_k)
                else:
                    buy_sell_topk_k = 1
                    buy_sell_topk = None

                # Adjust topk for add_modify_delete
                if 'add_modify_delete' in outputs:
                    add_modify_delete_probs = F.softmax(outputs['add_modify_delete'], dim=-1)
                    num_amd_classes = add_modify_delete_probs.size(-1)
                    amd_topk_k = min(beam_width, num_amd_classes)
                    amd_topk = torch.topk(add_modify_delete_probs, amd_topk_k)
                else:
                    amd_topk_k = 1
                    amd_topk = None

                # Adjust topk for distance
                if 'distance' in outputs:
                    distance_probs = F.softmax(outputs['distance'], dim=-1)
                    num_distance_classes = distance_probs.size(-1)
                    distance_topk_k = min(beam_width, num_distance_classes)
                    distance_topk = torch.topk(distance_probs, distance_topk_k)
                else:
                    distance_topk_k = 1
                    distance_topk = None

                # For continuous features, we can use the predicted value directly
                quantity_pred = outputs['quantity'].item() if 'quantity' in outputs else 0.0
                price_pred = outputs['price'].item() if 'price' in outputs else 0.0

                # Generate combinations of top predictions
                for i in range(buy_sell_topk_k):
                    for j in range(amd_topk_k):
                        for k in range(distance_topk_k):
                            new_seq = {key: seq[key] + [sample[key].item()] for key in seq}
                            if buy_sell_topk is not None:
                                new_seq['buy_sell'][-1] = buy_sell_topk.indices[0][i].item()
                            else:
                                new_seq['buy_sell'][-1] = seq['buy_sell'][-1]  # Use previous value

                            if amd_topk is not None:
                                new_seq['add_modify_delete'][-1] = amd_topk.indices[0][j].item()
                            else:
                                new_seq['add_modify_delete'][-1] = seq['add_modify_delete'][-1]  # Use previous value

                            if distance_topk is not None:
                                new_seq['distance'][-1] = distance_topk.indices[0][k].item()
                            else:
                                new_seq['distance'][-1] = seq['distance'][-1]  # Use previous value

                            new_seq['quantity'][-1] = quantity_pred
                            new_seq['price'][-1] = price_pred

                            # Compute new cumulative log probability
                            log_prob = cum_log_prob
                            if buy_sell_topk is not None:
                                log_prob += torch.log(buy_sell_topk.values[0][i] + 1e-9).item()
                            if amd_topk is not None:
                                log_prob += torch.log(amd_topk.values[0][j] + 1e-9).item()
                            if distance_topk is not None:
                                log_prob += torch.log(distance_topk.values[0][k] + 1e-9).item()

                            candidates.append((new_seq, log_prob))

                # Keep top beam_width candidates
                candidates.sort(key=lambda x: x[1], reverse=True)
                new_beam.extend(candidates[:beam_width])

            # Keep top beam_width sequences
            new_beam.sort(key=lambda x: x[1], reverse=True)
            beam = new_beam[:beam_width]

        # After generation, sample from the beam to get the required number of samples
        generated_sequences = [seq for seq, _ in beam[:num_samples]]
        return generated_sequences

# Example Usage
if __name__ == "__main__":
    # Training parameters
    training_params = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'num_epochs': 5,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }

    # Dataset parameters
    dataset_params = {
        'num_samples': 5000,
        'mask_prob': 0.15,
        'max_seq_len': 50
    }

    # Model parameters for OrderModel
    model_params = {
        'N_sec_ids': 100,
        'N_distance_bins': 10,
        'embedding_dim': 32,
        'hidden_dim': 64
    }

    # Train OrderModel
    print("Training OrderModel...")
#    model = train_model(OrderModel, model_params, dataset_params, training_params)

    # Example of training another model, e.g., LSTMModel
#    '''
    print("Training LSTMModel...")
    model_params_lstm = {
        'input_size': 8,  # Number of features in seq_data
        'hidden_size': 64,
        'num_layers': 2,
        'output_keys': ['security_id', 'buy_sell', 'add_modify_delete', 'distance', 'quantity', 'price']
    }
    model = train_model(LSTMModel, model_params_lstm, dataset_params, training_params)
#    '''

    # Generate sequences using beam search
    print("\nGenerating sequences using beam search...")
    # Prepare past_data
    past_data = {
        'timestamp': [1000, 1001, 1002],
        'position': [0, 1, 2],
        'security_id': [5, 5, 5],
        'buy_sell': [0, 1, 0],
        'add_modify_delete': [0, 1, 2],
        'quantity': [500.0, 600.0, 700.0],
        'price': [50.0, 51.0, 52.0],
        'distance': [3, 2, 1]
    }

    # Parameters for generation
    security_id = 5
    num_timesteps = 5
    beam_width = 3
    num_samples = 3

    generated_sequences = generate_sequences(
        model=model,
        past_data=past_data,
        security_id=security_id,
        num_timesteps=num_timesteps,
        beam_width=beam_width,
        num_samples=num_samples,
        device=training_params['device']
    )

    # Compute distribution of buys vs sells
    total_buy = 0
    total_sell = 0
    total_actions = 0

    for seq in generated_sequences:
        buy_sell_seq = seq['buy_sell'][-num_timesteps:]  # Only consider generated timesteps
        total_buy += buy_sell_seq.count(0)
        total_sell += buy_sell_seq.count(1)
        total_actions += len(buy_sell_seq)

    buy_percentage = (total_buy / total_actions) * 100 if total_actions > 0 else 0
    sell_percentage = (total_sell / total_actions) * 100 if total_actions > 0 else 0

    print(f"\nGenerated {num_samples} sequences of {num_timesteps} timesteps each.")
    print(f"Buy actions: {buy_percentage:.2f}%")
    print(f"Sell actions: {sell_percentage:.2f}%")

Training OrderModel...
Training LSTMModel...


<ipython-input-10-f82e91cc8d66>:449: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


RuntimeError: Expected floating point type for target with class probabilities, got Long

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random
from collections import defaultdict
import numpy as np

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
            'position': torch.arange(seq_len)  # Row number
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        # List of features to potentially mask
        maskable_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']

        # Decide how many features to mask (at least one)
        num_features_to_mask = random.randint(1, len(maskable_keys))

        # Randomly select features to mask
        features_to_mask = random.sample(maskable_keys, num_features_to_mask)

        for key in maskable_keys:
            if key in features_to_mask:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Learnable Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(PositionalEncoder, self).__init__()
        self.timestamp_encoder = nn.Linear(1, embedding_dim)
        self.position_encoder = nn.Embedding(1000, embedding_dim)  # Assume max 1000 positions

    def forward(self, timestamps, positions):
        timestamp_emb = self.timestamp_encoder(timestamps.unsqueeze(-1).float())
        position_emb = self.position_encoder(positions)
        return timestamp_emb + position_emb

# Base Model Class
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, *args, **kwargs):
        raise NotImplementedError

# Order Model with Positional Encoder
class OrderModel(BaseModel):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # Positional Encoder
        self.positional_encoder = PositionalEncoder(embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        batch_size = x['timestamp'].shape[0]

        # Process the masked sample
        timestamp = x['timestamp']
        position = x['position']
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        positional_emb = self.positional_encoder(timestamp, position)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([positional_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings

        seq_timestamp = seq_data['timestamp']
        seq_position = seq_data['position']
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_positional_emb = self.positional_encoder(seq_timestamp, seq_position)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_positional_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_emb, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Additional Models

class MatrixRegressionModel(BaseModel):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, input_size, output_keys, num_classes_dict):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(input_size, input_size))
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(input_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(input_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        x_transformed = x_vector @ self.A

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x_transformed)

        return outputs

class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, output_keys, num_classes_dict):
        super(LogisticRegressionModel, self).__init__()
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(input_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(input_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x_vector)
        return outputs

class CNNModel(BaseModel):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, input_size, output_keys, num_classes_dict):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(128, num_classes)
            else:
                self.output_heads[key] = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        seq_vector = seq_vector.permute(0, 2, 1)  # (batch_size, num_features, seq_len)
        x = self.relu(self.conv1(seq_vector))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x)

        return outputs

class LSTMModel(BaseModel):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, output_keys, num_classes_dict):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(hidden_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(hidden_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        packed_seq_emb = pack_padded_sequence(seq_vector, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        h0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)
        c0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)

        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb, (h0, c0))

        # Get the last hidden state
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_size)

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](seq_context)

        return outputs

class TransformerModel(BaseModel):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallelizable.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.
    """
    def __init__(self, input_size, num_heads, num_layers, output_keys, num_classes_dict):
        super(TransformerModel, self).__init__()
        self.positional_encoder = PositionalEncoder(input_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(input_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(input_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        # Add positional encoding
        positions = torch.arange(0, seq_vector.size(1), device=seq_vector.device).unsqueeze(0).repeat(seq_vector.size(0), 1)
        seq_vector = seq_vector + self.positional_encoder(positions, positions)

        # Transformer expects input shape (seq_len, batch_size, feature_dim)
        seq_vector = seq_vector.permute(1, 0, 2)
        transformer_output = self.transformer_encoder(seq_vector)
        # Take the output corresponding to the last position
        x = transformer_output[-1, :, :]
        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x)

        return outputs

# Collate function for DataLoader
def collate_fn(batch):
    # batch is a list of (masked_sample, target, sequence)
    batch_size = len(batch)
    masked_samples = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    sequences = [item[2] for item in batch]

    # Convert masked_samples to tensors
    batch_data = {}
    batch_target = {}
    seq_data = {}
    seq_lengths = []

    # Process data
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_data[key] = torch.tensor([sample[key] for sample in masked_samples])

    # Process sequences
    # Sequences are variable-length
    # For each key, we have a list of sequences
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        seq_list = [torch.tensor(seq[key]) for seq in sequences]
        seq_padded = nn.utils.rnn.pad_sequence(seq_list, batch_first=True, padding_value=0)
        seq_data[key] = seq_padded
    # Record lengths
    seq_lengths = [len(seq['timestamp']) for seq in sequences]

    # Convert seq_lengths to tensor
    seq_lengths = torch.tensor(seq_lengths, dtype=torch.long)

    # Process target (exclude 'timestamp' and 'position' as they're not in target)
    for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_target[key] = []
        for target in targets:
            if target[key] is not None:
                batch_target[key].append(target[key])
            else:
                batch_target[key].append(-100 if key not in ['quantity', 'price'] else 0.0)

        # Convert to tensor and create mask
        if key in ['quantity', 'price']:
            # For continuous features
            batch_target[key + '_mask'] = torch.tensor([t != 0.0 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.float)
        else:
            # For categorical features
            batch_target[key + '_mask'] = torch.tensor([t != -100 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.long)

    return batch_data, batch_target, seq_data, seq_lengths

# Training and Testing Loop
def train_model(model_class, model_params, dataset_params, training_params):
    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    # Create model
    model = model_class(**model_params).to(training_params['device'])
    optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

    # Loss functions
    criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
    criterion_l1 = nn.L1Loss()

    # Training loop
    for epoch in range(training_params['num_epochs']):
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for each feature
            for key in ['security_id', 'buy_sell', 'add_modify_delete', 'distance']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key]
                        loss_feature = criterion_ce(output, target)
                        loss += loss_feature

            for key in ['quantity', 'price']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key].squeeze()
                        loss_feature = criterion_l1(output[mask], target[mask])
                        loss += loss_feature

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

    return model

# Beam Search Generation Function (unchanged)
def generate_sequences(model, past_data, security_id, num_timesteps, beam_width, num_samples, device='cpu'):
    # [Function body remains the same]
    # For brevity, we'll keep the function as is
    pass

# Example Usage
if __name__ == "__main__":
    # Training parameters
    training_params = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'num_epochs': 2,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }

    # Dataset parameters
    dataset_params = {
        'num_samples': 5000,
        'mask_prob': 0.15,
        'max_seq_len': 50
    }

    # Common parameters
    output_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'distance', 'quantity', 'price']
    num_classes_dict = {
        'security_id': 100,
        'buy_sell': 2,
        'add_modify_delete': 3,
        'distance': 10
    }

    # Example 1: Training OrderModel
    print("Training OrderModel...")
    model_params_order = {
        'N_sec_ids': 100,
        'N_distance_bins': 10,
        'embedding_dim': 32,
        'hidden_dim': 64
    }
    model_order = train_model(OrderModel, model_params_order, dataset_params, training_params)

    # Example 2: Training LSTMModel
    print("\nTraining LSTMModel...")
    model_params_lstm = {
        'input_size': 8,  # Number of features in seq_data
        'hidden_size': 64,
        'num_layers': 2,
        'output_keys': output_keys,
        'num_classes_dict': num_classes_dict
    }
    model_lstm = train_model(LSTMModel, model_params_lstm, dataset_params, training_params)

    # Example 3: Training CNNModel
    print("\nTraining CNNModel...")
    model_params_cnn = {
        'input_size': 8,  # Number of features in seq_data
        'output_keys': output_keys,
        'num_classes_dict': num_classes_dict
    }
    model_cnn = train_model(CNNModel, model_params_cnn, dataset_params, training_params)

    # Example 4: Training LogisticRegressionModel
    print("\nTraining LogisticRegressionModel...")
    model_params_lr = {
        'input_size': 8,  # Number of features in batch_data
        'output_keys': output_keys,
        'num_classes_dict': num_classes_dict
    }
    model_lr = train_model(LogisticRegressionModel, model_params_lr, dataset_params, training_params)

    # Example 5: Training MatrixRegressionModel
    print("\nTraining MatrixRegressionModel...")
    model_params_mr = {
        'input_size': 8,  # Number of features in batch_data
        'output_keys': output_keys,
        'num_classes_dict': num_classes_dict
    }
    model_mr = train_model(MatrixRegressionModel, model_params_mr, dataset_params, training_params)

    # Example of generating sequences (for brevity, we use model_order)
    print("\nGenerating sequences using beam search with OrderModel...")
    # Prepare past_data
    past_data = {
        'timestamp': [1000, 1001, 1002],
        'position': [0, 1, 2],
        'security_id': [5, 5, 5],
        'buy_sell': [0, 1, 0],
        'add_modify_delete': [0, 1, 2],
        'quantity': [500.0, 600.0, 700.0],
        'price': [50.0, 51.0, 52.0],
        'distance': [3, 2, 1]
    }

    # Parameters for generation
    security_id = 5
    num_timesteps = 5
    beam_width = 3
    num_samples = 3

    generated_sequences = generate_sequences(
        model=model_order,
        past_data=past_data,
        security_id=security_id,
        num_timesteps=num_timesteps,
        beam_width=beam_width,
        num_samples=num_samples,
        device=training_params['device']
    )

    # Compute distribution of buys vs sells
    # For brevity, we will not reimplement the generate_sequences function here
    # Assume it works as in the previous code

    print("\nAll models have been trained and sequences generated successfully.")

Training OrderModel...


<ipython-input-11-6fa0b59fd028>:471: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 381.1270192140227
Epoch 2, Loss: 341.25743229374007

Training LSTMModel...
Epoch 1, Loss: 540.2828345815087
Epoch 2, Loss: 528.951018315212

Training CNNModel...
Epoch 1, Loss: 326.2109530503583
Epoch 2, Loss: 292.82250053259975

Training LogisticRegressionModel...
Epoch 1, Loss: 889.8038691623955
Epoch 2, Loss: 599.5396227016571

Training MatrixRegressionModel...
Epoch 1, Loss: 1410.0754884367536
Epoch 2, Loss: 593.8235143187699

Generating sequences using beam search with OrderModel...

All models have been trained and sequences generated successfully.


In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random
from collections import defaultdict
import numpy as np

# Define the Dataset
class MaskedOrderDataset(Dataset):
    def __init__(self, num_samples, mask_prob=0.15, max_seq_len=50):
        self.num_samples = num_samples
        self.N_sec_ids = 100  # Number of unique securities
        self.N_distance_bins = 10  # Number of distance bins
        self.mask_prob = mask_prob
        self.max_seq_len = max_seq_len

        # Generate synthetic data
        self.timestamps = torch.randint(0, 1000, (num_samples,))
        self.security_ids = torch.randint(0, self.N_sec_ids, (num_samples,))
        self.buy_sell = torch.randint(0, 2, (num_samples,))  # 0: buy, 1: sell
        self.add_modify_delete = torch.randint(0, 3, (num_samples,))  # 0: add, 1: modify, 2: delete
        self.quantity = torch.randint(1, 1000, (num_samples,)).float()
        self.price = torch.rand(num_samples) * 100  # Prices between 0 and 100
        self.distance = torch.randint(0, self.N_distance_bins, (num_samples,))

        # Sort the data by timestamps (ascending)
        sorted_indices = torch.argsort(self.timestamps)
        self.timestamps = self.timestamps[sorted_indices]
        self.security_ids = self.security_ids[sorted_indices]
        self.buy_sell = self.buy_sell[sorted_indices]
        self.add_modify_delete = self.add_modify_delete[sorted_indices]
        self.quantity = self.quantity[sorted_indices]
        self.price = self.price[sorted_indices]
        self.distance = self.distance[sorted_indices]

        # Define mask indices
        self.security_id_mask = self.N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = self.N_distance_bins

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Get sequence from idx to end, up to max_seq_len
        end_idx = min(idx + self.max_seq_len, self.num_samples)
        seq_len = end_idx - idx
        sequence = {
            'timestamp': self.timestamps[idx:end_idx],
            'security_id': self.security_ids[idx:end_idx],
            'buy_sell': self.buy_sell[idx:end_idx],
            'add_modify_delete': self.add_modify_delete[idx:end_idx],
            'quantity': self.quantity[idx:end_idx],
            'price': self.price[idx:end_idx],
            'distance': self.distance[idx:end_idx],
            'position': torch.arange(seq_len)  # Row number
        }

        # Apply masking logic to the first element (idx)
        sample = {key: sequence[key][0] for key in sequence}
        masked_sample = sample.copy()
        target = {}

        # List of features to potentially mask
        maskable_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']

        # Decide how many features to mask (at least one)
        num_features_to_mask = random.randint(1, len(maskable_keys))

        # Randomly select features to mask
        features_to_mask = random.sample(maskable_keys, num_features_to_mask)

        for key in maskable_keys:
            if key in features_to_mask:
                # Mask the feature
                target[key] = sample[key]
                if key == 'security_id':
                    masked_sample[key] = self.security_id_mask
                elif key == 'buy_sell':
                    masked_sample[key] = self.buy_sell_mask
                elif key == 'add_modify_delete':
                    masked_sample[key] = self.add_modify_delete_mask
                elif key == 'distance':
                    masked_sample[key] = self.distance_mask
                elif key in ['quantity', 'price']:
                    masked_sample[key] = 0.0  # For continuous features, use 0.0 as masked value
            else:
                target[key] = None  # Not masked

        # Return masked_sample, target, and the sequence starting from idx
        return masked_sample, target, sequence

# Learnable Positional Encoder
class PositionalEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(PositionalEncoder, self).__init__()
        self.timestamp_encoder = nn.Linear(1, embedding_dim)
        self.position_encoder = nn.Embedding(1000, embedding_dim)  # Assume max 1000 positions

    def forward(self, timestamps, positions):
        timestamp_emb = self.timestamp_encoder(timestamps.unsqueeze(-1).float())
        position_emb = self.position_encoder(positions)
        return timestamp_emb + position_emb

# Base Model Class
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, *args, **kwargs):
        raise NotImplementedError

# Order Model with Positional Encoder
class OrderModel(BaseModel):
    def __init__(self, N_sec_ids, N_distance_bins, embedding_dim=32, hidden_dim=64):
        super(OrderModel, self).__init__()

        # Mask indices
        self.security_id_mask = N_sec_ids
        self.buy_sell_mask = 2
        self.add_modify_delete_mask = 3
        self.distance_mask = N_distance_bins

        # Embedding layers for categorical features
        self.security_id_embedding = nn.Embedding(N_sec_ids + 1, embedding_dim, padding_idx=self.security_id_mask)
        self.buy_sell_embedding = nn.Embedding(3, embedding_dim, padding_idx=self.buy_sell_mask)  # 0,1,2(mask)
        self.add_modify_delete_embedding = nn.Embedding(4, embedding_dim, padding_idx=self.add_modify_delete_mask)  # 0,1,2,3(mask)
        self.distance_embedding = nn.Embedding(N_distance_bins + 1, embedding_dim, padding_idx=self.distance_mask)

        # Linear layers for continuous features
        self.quantity_linear = nn.Linear(1, embedding_dim)
        self.price_linear = nn.Linear(1, embedding_dim)

        # Positional Encoder
        self.positional_encoder = PositionalEncoder(embedding_dim)

        # LSTM for sequences
        self.lstm_input_dim = embedding_dim * 7  # Number of features
        self.lstm = nn.LSTM(input_size=self.lstm_input_dim, hidden_size=hidden_dim, batch_first=True)

        # Final fully connected layers
        self.fc1 = nn.Linear(embedding_dim * 7 + hidden_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.relu = nn.ReLU()

        # Output heads
        self.security_id_head = nn.Linear(128, N_sec_ids)
        self.buy_sell_head = nn.Linear(128, 2)
        self.add_modify_delete_head = nn.Linear(128, 3)
        self.distance_head = nn.Linear(128, N_distance_bins)
        self.quantity_head = nn.Linear(128, 1)
        self.price_head = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x is a dictionary of features for the masked sample
        # seq_data is a dictionary of sequences
        # seq_lengths is a list of sequence lengths

        batch_size = x['timestamp'].shape[0]

        # Process the masked sample
        timestamp = x['timestamp']
        position = x['position']
        security_id = x['security_id']
        buy_sell = x['buy_sell']
        add_modify_delete = x['add_modify_delete']
        quantity = x['quantity'].unsqueeze(1)
        price = x['price'].unsqueeze(1)
        distance = x['distance']

        # Embeddings
        positional_emb = self.positional_encoder(timestamp, position)
        security_id_emb = self.security_id_embedding(security_id)
        buy_sell_emb = self.buy_sell_embedding(buy_sell)
        add_modify_delete_emb = self.add_modify_delete_embedding(add_modify_delete)
        quantity_emb = self.quantity_linear(quantity)
        price_emb = self.price_linear(price)
        distance_emb = self.distance_embedding(distance)

        # Concatenate embeddings
        sample_emb = torch.cat([positional_emb,
                                security_id_emb,
                                buy_sell_emb,
                                add_modify_delete_emb,
                                quantity_emb,
                                price_emb,
                                distance_emb], dim=1)  # Shape: (batch_size, embedding_dim * 7)

        # Process the sequence data
        # For each feature in seq_data, get embeddings

        seq_timestamp = seq_data['timestamp']
        seq_position = seq_data['position']
        seq_security_id = seq_data['security_id']
        seq_buy_sell = seq_data['buy_sell']
        seq_add_modify_delete = seq_data['add_modify_delete']
        seq_quantity = seq_data['quantity']
        seq_price = seq_data['price']
        seq_distance = seq_data['distance']

        # Embeddings for sequence data
        seq_positional_emb = self.positional_encoder(seq_timestamp, seq_position)
        seq_security_id_emb = self.security_id_embedding(seq_security_id)
        seq_buy_sell_emb = self.buy_sell_embedding(seq_buy_sell)
        seq_add_modify_delete_emb = self.add_modify_delete_embedding(seq_add_modify_delete)
        seq_quantity_emb = self.quantity_linear(seq_quantity.unsqueeze(-1))
        seq_price_emb = self.price_linear(seq_price.unsqueeze(-1))
        seq_distance_emb = self.distance_embedding(seq_distance)

        # Concatenate sequence embeddings
        seq_emb = torch.cat([seq_positional_emb,
                             seq_security_id_emb,
                             seq_buy_sell_emb,
                             seq_add_modify_delete_emb,
                             seq_quantity_emb,
                             seq_price_emb,
                             seq_distance_emb], dim=2)  # Shape: (batch_size, seq_len, embedding_dim * 7)

        # Pack the sequences
        packed_seq_emb = pack_padded_sequence(seq_emb, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb)

        # Get the last hidden state for each sequence
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_dim)

        # Combine the sample embedding with the sequence context
        combined = torch.cat([sample_emb, seq_context], dim=1)  # Shape: (batch_size, embedding_dim * 7 + hidden_dim)

        # Forward pass
        x = self.fc1(combined)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        # Outputs
        outputs = {
            'security_id': self.security_id_head(x),
            'buy_sell': self.buy_sell_head(x),
            'add_modify_delete': self.add_modify_delete_head(x),
            'distance': self.distance_head(x),
            'quantity': self.quantity_head(x),
            'price': self.price_head(x)
        }

        return outputs

# Additional Models

class MatrixRegressionModel(BaseModel):
    """
    Matrix Regression Model: Y = A * X * B

    Advantages:
    - Simple and interpretable.
    - Fast training.

    Disadvantages:
    - Limited in capturing complex patterns.
    """
    def __init__(self, input_size, output_keys, num_classes_dict):
        super(MatrixRegressionModel, self).__init__()
        self.A = nn.Parameter(torch.randn(input_size, input_size))
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in self.output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(input_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(input_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        x_transformed = x_vector @ self.A

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x_transformed)

        return outputs

class LogisticRegressionModel(BaseModel):
    """
    Logistic Regression Model.

    Advantages:
    - Simple and interpretable.
    - Good baseline model.

    Disadvantages:
    - Assumes linear relationship.
    - May underfit complex data.
    """
    def __init__(self, input_size, output_keys, num_classes_dict):
        super(LogisticRegressionModel, self).__init__()
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in self.output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(input_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(input_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # x: batch_data concatenated into a single vector
        x_vector = torch.cat([x[key].unsqueeze(-1).float() for key in x], dim=1)
        x_vector = x_vector.view(x_vector.size(0), -1)
        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x_vector)
        return outputs

class CNNModel(BaseModel):
    """
    Convolutional Neural Network Model.

    Advantages:
    - Captures local temporal patterns.
    - Efficient computation.

    Disadvantages:
    - Limited in capturing long-term dependencies.
    """
    def __init__(self, input_size, output_keys, num_classes_dict):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(128, num_classes)
            else:
                self.output_heads[key] = nn.Linear(128, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        seq_vector = seq_vector.permute(0, 2, 1)  # (batch_size, num_features, seq_len)
        x = self.relu(self.conv1(seq_vector))
        x = self.relu(self.conv2(x))
        x = torch.mean(x, dim=2)  # Global average pooling

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x)

        return outputs

class LSTMModel(BaseModel):
    """
    Long Short-Term Memory Model.

    Advantages:
    - Captures long-term dependencies.
    - Suitable for sequential data.

    Disadvantages:
    - Computationally intensive.
    - Prone to overfitting.
    """
    def __init__(self, input_size, hidden_size, num_layers, output_keys, num_classes_dict):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in self.output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(hidden_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(hidden_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        packed_seq_emb = pack_padded_sequence(seq_vector, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)

        h0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)
        c0 = torch.zeros(self.lstm.num_layers, seq_vector.size(0), self.lstm.hidden_size).to(seq_vector.device)

        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb, (h0, c0))

        # Get the last hidden state
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_size)

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](seq_context)

        return outputs

class TransformerModel(BaseModel):
    """
    Transformer Model.

    Advantages:
    - Captures global dependencies.
    - Parallelizable.

    Disadvantages:
    - Requires large datasets.
    - Computationally intensive.
    """
    def __init__(self, input_size, num_heads, num_layers, output_keys, num_classes_dict):
        super(TransformerModel, self).__init__()
        self.input_size = input_size
        self.positional_encoder = PositionalEncoder(input_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in self.output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(input_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(input_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        # Add positional encoding
        positions = torch.arange(0, seq_vector.size(1), device=seq_vector.device).unsqueeze(0).repeat(seq_vector.size(0), 1)
        seq_vector = seq_vector + self.positional_encoder(positions, positions)

        # Transformer expects input shape (seq_len, batch_size, feature_dim)
        seq_vector = seq_vector.permute(1, 0, 2)
        transformer_output = self.transformer_encoder(seq_vector)
        # Take the output corresponding to the last position
        x = transformer_output[-1, :, :]
        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](x)

        return outputs

class MambaModel(BaseModel):
    """
    Mamba Model.

    Advantages:
    - Custom model that combines CNN and LSTM architectures.
    - Captures both local and long-term dependencies.

    Disadvantages:
    - More complex and may require careful tuning.
    """
    def __init__(self, input_size, hidden_size, num_layers, output_keys, num_classes_dict):
        super(MambaModel, self).__init__()
        # CNN part
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # LSTM part
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

        # Output heads
        self.output_heads = nn.ModuleDict()
        self.output_keys = output_keys
        for key in self.output_keys:
            if key in num_classes_dict:
                num_classes = num_classes_dict[key]
                self.output_heads[key] = nn.Linear(hidden_size, num_classes)
            else:
                self.output_heads[key] = nn.Linear(hidden_size, 1)

    def forward(self, x, seq_data, seq_lengths):
        # seq_data: concatenated features
        seq_vector = torch.cat([seq_data[key].unsqueeze(-1).float() for key in seq_data], dim=2)
        seq_vector = seq_vector.permute(0, 2, 1)  # (batch_size, num_features, seq_len)

        # CNN part
        x = self.relu(self.conv1(seq_vector))
        x = self.relu(self.conv2(x))
        x = x.permute(0, 2, 1)  # (batch_size, seq_len, channels)

        # LSTM part
        packed_seq_emb = pack_padded_sequence(x, seq_lengths.cpu(), batch_first=True, enforce_sorted=False)
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        packed_output, (h_n, c_n) = self.lstm(packed_seq_emb, (h0, c0))
        seq_context = h_n[-1]  # Shape: (batch_size, hidden_size)

        outputs = {}
        for key in self.output_keys:
            outputs[key] = self.output_heads[key](seq_context)

        return outputs

# Collate function for DataLoader
def collate_fn(batch):
    # batch is a list of (masked_sample, target, sequence)
    batch_size = len(batch)
    masked_samples = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    sequences = [item[2] for item in batch]

    # Convert masked_samples to tensors
    batch_data = {}
    batch_target = {}
    seq_data = {}
    seq_lengths = []

    # Process data
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_data[key] = torch.tensor([sample[key] for sample in masked_samples])

    # Process sequences
    # Sequences are variable-length
    # For each key, we have a list of sequences
    for key in ['timestamp', 'position', 'security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        seq_list = [torch.tensor(seq[key]) for seq in sequences]
        seq_padded = nn.utils.rnn.pad_sequence(seq_list, batch_first=True, padding_value=0)
        seq_data[key] = seq_padded
    # Record lengths
    seq_lengths = [len(seq['timestamp']) for seq in sequences]

    # Convert seq_lengths to tensor
    seq_lengths = torch.tensor(seq_lengths, dtype=torch.long)

    # Process target (exclude 'timestamp' and 'position' as they're not in target)
    for key in ['security_id', 'buy_sell', 'add_modify_delete', 'quantity', 'price', 'distance']:
        batch_target[key] = []
        for target in targets:
            if target[key] is not None:
                batch_target[key].append(target[key])
            else:
                batch_target[key].append(-100 if key not in ['quantity', 'price'] else 0.0)

        # Convert to tensor and create mask
        if key in ['quantity', 'price']:
            # For continuous features
            batch_target[key + '_mask'] = torch.tensor([t != 0.0 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.float)
        else:
            # For categorical features
            batch_target[key + '_mask'] = torch.tensor([t != -100 for t in batch_target[key]], dtype=torch.bool)
            batch_target[key] = torch.tensor(batch_target[key], dtype=torch.long)

    return batch_data, batch_target, seq_data, seq_lengths

# Training and Testing Loop
def train_model(model_class, model_params, dataset_params, training_params):
    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    # Create model
    model = model_class(**model_params).to(training_params['device'])
    optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

    # Loss functions
    criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
    criterion_l1 = nn.L1Loss()

    # Training loop
    for epoch in range(training_params['num_epochs']):
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for each feature
            for key in ['security_id', 'buy_sell', 'add_modify_delete', 'distance']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key]
                        loss_feature = criterion_ce(output, target)
                        loss += loss_feature

            for key in ['quantity', 'price']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key].squeeze()
                        loss_feature = criterion_l1(output[mask], target[mask])
                        loss += loss_feature

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

    return model

# Beam Search Generation Function
def generate_sequences(model, past_data, security_id, num_timesteps, beam_width, num_samples, device='cpu'):
    """
    Generate sequences using beam search.

    Args:
        model: Trained model.
        past_data: Dictionary containing past sequences.
        security_id: The security ID to generate sequences for.
        num_timesteps: Number of timesteps to generate.
        beam_width: Beam width.
        num_samples: Number of samples to generate.
        device: Device to run the computations on.

    Returns:
        generated_sequences: A list of generated sequences.
    """
    model.eval()
    with torch.no_grad():
        # Initialize the beam with the past_data
        # Each beam entry is a tuple (sequence, cumulative_log_prob)
        beam = [({'timestamp': past_data['timestamp'],
                  'position': past_data['position'],
                  'security_id': past_data['security_id'],
                  'buy_sell': past_data['buy_sell'],
                  'add_modify_delete': past_data['add_modify_delete'],
                  'quantity': past_data['quantity'],
                  'price': past_data['price'],
                  'distance': past_data['distance']},
                 0.0)]  # Start with log probability 0

        for t in range(num_timesteps):
            new_beam = []
            for seq, cum_log_prob in beam:
                # Prepare input data
                # Use the last max_seq_len elements
                seq_len = len(seq['timestamp'])
                start_idx = max(0, seq_len - model.lstm_input_dim) if hasattr(model, 'lstm_input_dim') else 0
                seq_data = {key: torch.tensor(seq[key][start_idx:]).unsqueeze(0).to(device) for key in seq}
                seq_lengths = torch.tensor([seq_data['timestamp'].shape[1]]).to(device)

                # Create a sample with masked features (except security_id and timestamp)
                sample = {
                    'timestamp': torch.tensor([seq['timestamp'][-1] + 1]).to(device),
                    'position': torch.tensor([seq['position'][-1] + 1]).to(device),
                    'security_id': torch.tensor([security_id]).to(device),
                    'buy_sell': torch.tensor([model.buy_sell_mask if hasattr(model, 'buy_sell_mask') else 2]).to(device),
                    'add_modify_delete': torch.tensor([model.add_modify_delete_mask if hasattr(model, 'add_modify_delete_mask') else 3]).to(device),
                    'quantity': torch.tensor([0.0]).to(device),
                    'price': torch.tensor([0.0]).to(device),
                    'distance': torch.tensor([model.distance_mask if hasattr(model, 'distance_mask') else 10]).to(device)
                }

                # Predict missing features
                outputs = model(sample, seq_data, seq_lengths)

                # For each categorical feature, get top N predictions
                candidates = []

                # Adjust topk for buy_sell
                if 'buy_sell' in outputs:
                    buy_sell_probs = F.softmax(outputs['buy_sell'], dim=-1)
                    num_buy_sell_classes = buy_sell_probs.size(-1)
                    buy_sell_topk_k = min(beam_width, num_buy_sell_classes)
                    buy_sell_topk = torch.topk(buy_sell_probs, buy_sell_topk_k)
                else:
                    buy_sell_topk_k = 1
                    buy_sell_topk = None

                # Adjust topk for add_modify_delete
                if 'add_modify_delete' in outputs:
                    add_modify_delete_probs = F.softmax(outputs['add_modify_delete'], dim=-1)
                    num_amd_classes = add_modify_delete_probs.size(-1)
                    amd_topk_k = min(beam_width, num_amd_classes)
                    amd_topk = torch.topk(add_modify_delete_probs, amd_topk_k)
                else:
                    amd_topk_k = 1
                    amd_topk = None

                # Adjust topk for distance
                if 'distance' in outputs:
                    distance_probs = F.softmax(outputs['distance'], dim=-1)
                    num_distance_classes = distance_probs.size(-1)
                    distance_topk_k = min(beam_width, num_distance_classes)
                    distance_topk = torch.topk(distance_probs, distance_topk_k)
                else:
                    distance_topk_k = 1
                    distance_topk = None

                # For continuous features, we can use the predicted value directly
                quantity_pred = outputs['quantity'].item() if 'quantity' in outputs else 0.0
                price_pred = outputs['price'].item() if 'price' in outputs else 0.0

                # Generate combinations of top predictions
                for i in range(buy_sell_topk_k):
                    for j in range(amd_topk_k):
                        for k in range(distance_topk_k):
                            new_seq = {key: seq[key] + [sample[key].item()] for key in seq}
                            if buy_sell_topk is not None:
                                new_seq['buy_sell'][-1] = buy_sell_topk.indices[0][i].item()
                            else:
                                new_seq['buy_sell'][-1] = seq['buy_sell'][-1]  # Use previous value

                            if amd_topk is not None:
                                new_seq['add_modify_delete'][-1] = amd_topk.indices[0][j].item()
                            else:
                                new_seq['add_modify_delete'][-1] = seq['add_modify_delete'][-1]  # Use previous value

                            if distance_topk is not None:
                                new_seq['distance'][-1] = distance_topk.indices[0][k].item()
                            else:
                                new_seq['distance'][-1] = seq['distance'][-1]  # Use previous value

                            new_seq['quantity'][-1] = quantity_pred
                            new_seq['price'][-1] = price_pred

                            # Compute new cumulative log probability
                            log_prob = cum_log_prob
                            if buy_sell_topk is not None:
                                log_prob += torch.log(buy_sell_topk.values[0][i] + 1e-9).item()
                            if amd_topk is not None:
                                log_prob += torch.log(amd_topk.values[0][j] + 1e-9).item()
                            if distance_topk is not None:
                                log_prob += torch.log(distance_topk.values[0][k] + 1e-9).item()

                            candidates.append((new_seq, log_prob))

                # Keep top beam_width candidates
                candidates.sort(key=lambda x: x[1], reverse=True)
                new_beam.extend(candidates[:beam_width])

            # Keep top beam_width sequences
            new_beam.sort(key=lambda x: x[1], reverse=True)
            beam = new_beam[:beam_width]

        # After generation, sample from the beam to get the required number of samples
        generated_sequences = [seq for seq, _ in beam[:num_samples]]
        return generated_sequences

# Hyperparameter Optimization
def hyperparameter_optimization(model_class, model_params_grid, dataset_params, training_params, output_keys, num_classes_dict):
    from sklearn.model_selection import ParameterGrid
    import copy

    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    best_model = None
    best_loss = float('inf')
    best_params = None

    param_grid = list(ParameterGrid(model_params_grid))

    for params in param_grid:
        print(f"Testing parameters: {params}")
        model_params = params.copy()
        model_params['output_keys'] = output_keys
        model_params['num_classes_dict'] = num_classes_dict

        model = model_class(**model_params).to(training_params['device'])
        optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

        # Loss functions
        criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
        criterion_l1 = nn.L1Loss()

        # Training loop
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for each feature
            for key in ['security_id', 'buy_sell', 'add_modify_delete', 'distance']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key]
                        loss_feature = criterion_ce(output, target)
                        loss += loss_feature

            for key in ['quantity', 'price']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key].squeeze()
                        loss_feature = criterion_l1(output[mask], target[mask])
                        loss += loss_feature

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Average Loss: {avg_loss}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model = copy.deepcopy(model)
            best_params = params

    print(f"Best Parameters: {best_params}")
    print(f"Best Loss: {best_loss}")

    return best_model

# Example Usage
if __name__ == "__main__":
    # Training parameters
    training_params = {
        'batch_size': 32,
        'learning_rate': 0.001,
        'num_epochs': 2,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }

    # Dataset parameters
    dataset_params = {
        'num_samples': 5000,
        'mask_prob': 0.15,
        'max_seq_len': 50
    }

    # Common parameters
    output_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'distance', 'quantity', 'price']
    num_classes_dict = {
        'security_id': 100,
        'buy_sell': 2,
        'add_modify_delete': 3,
        'distance': 10
    }

    # Example 1: Training TransformerModel
    print("Training TransformerModel...")
    model_params_transformer = {
        'input_size': 8,  # Number of features in seq_data
        'num_heads': 4,
        'num_layers': 2,
        'output_keys': output_keys,
        'num_classes_dict': num_classes_dict
    }
    model_transformer = train_model(TransformerModel, model_params_transformer, dataset_params, training_params)

    # Example 2: Training MambaModel
    print("\nTraining MambaModel...")
    model_params_mamba = {
        'input_size': 8,  # Number of features in seq_data
        'hidden_size': 64,
        'num_layers': 2,
        'output_keys': output_keys,
        'num_classes_dict': num_classes_dict
    }
    model_mamba = train_model(MambaModel, model_params_mamba, dataset_params, training_params)

    # Generate sequences using beam search with TransformerModel
    print("\nGenerating sequences using beam search with TransformerModel...")
    # Prepare past_data
    past_data = {
        'timestamp': [1000, 1001, 1002],
        'position': [0, 1, 2],
        'security_id': [5, 5, 5],
        'buy_sell': [0, 1, 0],
        'add_modify_delete': [0, 1, 2],
        'quantity': [500.0, 600.0, 700.0],
        'price': [50.0, 51.0, 52.0],
        'distance': [3, 2, 1]
    }

    # Parameters for generation
    security_id = 5
    num_timesteps = 5
    beam_width = 3
    num_samples = 3

    generated_sequences = generate_sequences(
        model=model_transformer,
        past_data=past_data,
        security_id=security_id,
        num_timesteps=num_timesteps,
        beam_width=beam_width,
        num_samples=num_samples,
        device=training_params['device']
    )

    # Compute distribution of buys vs sells
    total_buy = 0
    total_sell = 0
    total_actions = 0

    for seq in generated_sequences:
        buy_sell_seq = seq['buy_sell'][-num_timesteps:]  # Only consider generated timesteps
        total_buy += buy_sell_seq.count(0)
        total_sell += buy_sell_seq.count(1)
        total_actions += len(buy_sell_seq)

    buy_percentage = (total_buy / total_actions) * 100 if total_actions > 0 else 0
    sell_percentage = (total_sell / total_actions) * 100 if total_actions > 0 else 0

    print(f"\nGenerated {num_samples} sequences of {num_timesteps} timesteps each.")
    print(f"Buy actions: {buy_percentage:.2f}%")
    print(f"Sell actions: {sell_percentage:.2f}%")

    # Hyperparameter Optimization Example with MambaModel
    print("\nStarting Hyperparameter Optimization for MambaModel...")
    model_params_grid = {
        'hidden_size': [32, 64],
        'num_layers': [1, 2],
    }
    best_mamba_model = hyperparameter_optimization(
        model_class=MambaModel,
        model_params_grid=model_params_grid,
        dataset_params=dataset_params,
        training_params=training_params,
        output_keys=output_keys,
        num_classes_dict=num_classes_dict
    )

    print("\nHyperparameter optimization completed.")

Training TransformerModel...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 559.9849175131245
Epoch 2, Loss: 550.8923345675134

Training MambaModel...
Epoch 1, Loss: 540.1169313078474
Epoch 2, Loss: 519.5057787075165

Generating sequences using beam search with TransformerModel...

Generated 3 sequences of 5 timesteps each.
Buy actions: 0.00%
Sell actions: 100.00%

Starting Hyperparameter Optimization for MambaModel...
Testing parameters: {'hidden_size': 32, 'num_layers': 1}


TypeError: MambaModel.__init__() missing 1 required positional argument: 'input_size'

In [16]:
def hyperparameter_optimization(model_class, model_params_grid, dataset_params, training_params, output_keys, num_classes_dict):
    from sklearn.model_selection import ParameterGrid
    import copy

    # Create Dataset and DataLoader
    dataset = MaskedOrderDataset(**dataset_params)
    dataloader = DataLoader(dataset, batch_size=training_params['batch_size'], shuffle=True, collate_fn=collate_fn)

    best_model = None
    best_loss = float('inf')
    best_params = None

    param_grid = list(ParameterGrid(model_params_grid))

    for params in param_grid:
        print(f"Testing parameters: {params}")
        model_params = params.copy()
        model_params['input_size'] = 8  # Number of features in seq_data
        model_params['output_keys'] = output_keys
        model_params['num_classes_dict'] = num_classes_dict

        model = model_class(**model_params).to(training_params['device'])
        optimizer = optim.Adam(model.parameters(), lr=training_params['learning_rate'])

        # Loss functions
        criterion_ce = nn.CrossEntropyLoss(ignore_index=-100)
        criterion_l1 = nn.L1Loss()

        # Training loop
        model.train()
        total_loss = 0.0
        for batch_data, batch_target, seq_data, seq_lengths in dataloader:
            optimizer.zero_grad()

            # Move data to device
            device = training_params.get('device', 'cpu')
            batch_data = {k: v.to(device) for k, v in batch_data.items()}
            batch_target = {k: v.to(device) for k, v in batch_target.items()}
            seq_data = {k: v.to(device) for k, v in seq_data.items()}
            seq_lengths = seq_lengths.to(device)

            outputs = model(batch_data, seq_data, seq_lengths)

            loss = 0.0

            # Compute loss for each feature
            for key in ['security_id', 'buy_sell', 'add_modify_delete', 'distance']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key]
                        loss_feature = criterion_ce(output, target)
                        loss += loss_feature

            for key in ['quantity', 'price']:
                if key in outputs:
                    target = batch_target[key]
                    mask = batch_target[key + '_mask']
                    if mask.any():
                        output = outputs[key].squeeze()
                        loss_feature = criterion_l1(output[mask], target[mask])
                        loss += loss_feature

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Average Loss: {avg_loss}")

        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model = copy.deepcopy(model)
            best_params = params

    print(f"Best Parameters: {best_params}")
    print(f"Best Loss: {best_loss}")

    return best_model

In [17]:
# Training parameters
training_params = {
    'batch_size': 32,
    'learning_rate': 0.001,
    'num_epochs': 2,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Dataset parameters
dataset_params = {
    'num_samples': 5000,
    'mask_prob': 0.15,
    'max_seq_len': 50
}

# Common parameters
output_keys = ['security_id', 'buy_sell', 'add_modify_delete', 'distance', 'quantity', 'price']
num_classes_dict = {
    'security_id': 100,
    'buy_sell': 2,
    'add_modify_delete': 3,
    'distance': 10
}

In [18]:
print("\nTraining LogisticRegressionModel...")
model_params_lr = {
    'input_size': 8,  # Number of features in batch_data
    'output_keys': output_keys,
    'num_classes_dict': num_classes_dict
}
model_lr = train_model(LogisticRegressionModel, model_params_lr, dataset_params, training_params)


Training LogisticRegressionModel...


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 1038.635112981128
Epoch 2, Loss: 618.0137916127587


In [19]:
print("\nTraining MatrixRegressionModel...")
model_params_mr = {
    'input_size': 8,  # Number of features in batch_data
    'output_keys': output_keys,
    'num_classes_dict': num_classes_dict
}
model_mr = train_model(MatrixRegressionModel, model_params_mr, dataset_params, training_params)


Training MatrixRegressionModel...


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 1354.9664022846587
Epoch 2, Loss: 639.6293430206882


In [20]:
print("\nHyperparameter Optimization for MatrixRegressionModel...")
model_params_grid_mr = {
    'input_size': [8],  # Usually fixed
}
best_mr_model = hyperparameter_optimization(
    model_class=MatrixRegressionModel,
    model_params_grid=model_params_grid_mr,
    dataset_params=dataset_params,
    training_params=training_params,
    output_keys=output_keys,
    num_classes_dict=num_classes_dict
)


Hyperparameter Optimization for MatrixRegressionModel...
Testing parameters: {'input_size': 8}


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Average Loss: 1549.875912806031
Best Parameters: {'input_size': 8}
Best Loss: 1549.875912806031


In [21]:
print("\nTraining CNNModel...")
model_params_cnn = {
    'input_size': 8,  # Number of features in seq_data
    'output_keys': output_keys,
    'num_classes_dict': num_classes_dict
}
model_cnn = train_model(CNNModel, model_params_cnn, dataset_params, training_params)


Training CNNModel...


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 326.69811331390576
Epoch 2, Loss: 298.0500039264655


In [22]:
print("\nHyperparameter Optimization for CNNModel...")
model_params_grid_cnn = {
    'input_size': [8],  # Usually fixed
}
best_cnn_model = hyperparameter_optimization(
    model_class=CNNModel,
    model_params_grid=model_params_grid_cnn,
    dataset_params=dataset_params,
    training_params=training_params,
    output_keys=output_keys,
    num_classes_dict=num_classes_dict
)


Hyperparameter Optimization for CNNModel...
Testing parameters: {'input_size': 8}


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Average Loss: 333.3192240721101
Best Parameters: {'input_size': 8}
Best Loss: 333.3192240721101


In [23]:
print("\nTraining LSTMModel...")
model_params_lstm = {
    'input_size': 8,  # Number of features in seq_data
    'hidden_size': 64,
    'num_layers': 2,
    'output_keys': output_keys,
    'num_classes_dict': num_classes_dict
}
model_lstm = train_model(LSTMModel, model_params_lstm, dataset_params, training_params)


Training LSTMModel...


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 541.4942780512913
Epoch 2, Loss: 521.8279852411549


In [24]:
print("\nHyperparameter Optimization for LSTMModel...")
model_params_grid_lstm = {
    'hidden_size': [32, 64],
    'num_layers': [1, 2],
}
best_lstm_model = hyperparameter_optimization(
    model_class=LSTMModel,
    model_params_grid=model_params_grid_lstm,
    dataset_params=dataset_params,
    training_params=training_params,
    output_keys=output_keys,
    num_classes_dict=num_classes_dict
)


Hyperparameter Optimization for LSTMModel...
Testing parameters: {'hidden_size': 32, 'num_layers': 1}


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Average Loss: 556.6154732673791
Testing parameters: {'hidden_size': 32, 'num_layers': 2}
Average Loss: 553.4682630794064
Testing parameters: {'hidden_size': 64, 'num_layers': 1}
Average Loss: 549.3058979010126
Testing parameters: {'hidden_size': 64, 'num_layers': 2}
Average Loss: 543.2973115763087
Best Parameters: {'hidden_size': 64, 'num_layers': 2}
Best Loss: 543.2973115763087


In [25]:
print("\nTraining TransformerModel...")
model_params_transformer = {
    'input_size': 8,  # Number of features in seq_data
    'num_heads': 4,
    'num_layers': 2,
    'output_keys': output_keys,
    'num_classes_dict': num_classes_dict
}
model_transformer = train_model(TransformerModel, model_params_transformer, dataset_params, training_params)


Training TransformerModel...


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Epoch 1, Loss: 557.859285196681
Epoch 2, Loss: 550.991188195101


In [ ]:
print("\nHyperparameter Optimization for TransformerModel...")
model_params_grid_transformer = {
    'num_heads': [2, 4],
    'num_layers': [1, 2],
}
best_transformer_model = hyperparameter_optimization(
    model_class=TransformerModel,
    model_params_grid=model_params_grid_transformer,
    dataset_params=dataset_params,
    training_params=training_params,
    output_keys=output_keys,
    num_classes_dict=num_classes_dict
)


Hyperparameter Optimization for TransformerModel...
Testing parameters: {'num_heads': 2, 'num_layers': 1}


<ipython-input-12-79da02321f52>:526: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  seq_list = [torch.tensor(seq[key]) for seq in sequences]


Average Loss: 552.7555909369402
Testing parameters: {'num_heads': 2, 'num_layers': 2}
Average Loss: 558.3002187157892
Testing parameters: {'num_heads': 4, 'num_layers': 1}
